<div class="span5 alert alert-success">
<p> This workbook creates a dataset consisting of zip codes with windfarms and zip codes without windfarms. 
Also, a column that designates if the zipcode does or doesn't have a windfarm is added, along with a new feature representing education level </p>
</div>

<div class="span5 alert alert-success">
<p> Prepare </p>
</div>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Imports
import pandas as pd
from uszipcode import SearchEngine
import geopy.distance
import numpy as np

In [3]:
#Load the zip code database (https://pypi.org/project/uszipcode/)
search = SearchEngine(simple_zipcode=False)

In [4]:
#function to calculate distance between geocodes - this will be used to obtain zipcodes without windfarms located approximately
#25 miles from a zipcode with a windfarm

def distance(lat1, long1, lat2, long2):
    coords_1 = (lat1, long1)
    coords_2 = (lat2, long2)
    
    d = geopy.distance.distance(coords_1, coords_2).miles
            
    return d

In [5]:
#Load the cleaned zillow csv file to a dataframe
%cd C:\Users\Ken\Documents\KenP\Applications-DataScience\SpringboardCourseWork\CapstoneProject1Repository\09 CapstoneProject1Data\ZillowData

df_zi = pd.read_csv('project1_housing_data.csv')

df_zi = df_zi.sort_values(by=['ZipCode'])

df_zi.head(1)

C:\Users\Ken\Documents\KenP\Applications-DataScience\SpringboardCourseWork\CapstoneProject1Repository\09 CapstoneProject1Data\ZillowData


,Unnamed: 0,ZipCode,StateName,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,...,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08
3082,3082,1001,Massachusetts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,198400.0,190500.0,166700.0,163700.0,155200.0,177100.0,182800.0,198000.0,202700.0,NaN


In [6]:
#Load the merged windfarm and zillow csv file to a dataframe
%cd C:\Users\Ken\Documents\KenP\Applications-DataScience\SpringboardCourseWork\CapstoneProject1Repository\09 CapstoneProject1Data\MergedData

df_me = pd.read_csv('merged_wi_and_zi.csv')

df_me.head(1)

C:\Users\Ken\Documents\KenP\Applications-DataScience\SpringboardCourseWork\CapstoneProject1Repository\09 CapstoneProject1Data\MergedData


,Unnamed: 0,state,county,fips,project_name,project_op_year,windmills_count,capacity,longitude,latitude,...,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08
0,0,MA,Worcester County,25027,Nature's Classroom Wind Turbine,2009,1,0.1,-72.003899,42.110889,...,290300.0,279200.0,276000.0,252000.0,244500.0,259900.0,287800.0,295100.0,289100.0,NaN


<div class="span5 alert alert-success">
<p> Create the dataframe for zipcodes WITH windfarms. </p>
</div>

In [7]:
#Filter merged dataframe from 13 states to 5 (CA, MA, MN, OH, RI).  Then filter out all the duplicate zipcode records by
#keeping only the first record of each zipcode (note: each row in the file is a single windmill in a wind farm)

df_me2 = df_me[(df_me.state == 'CA') | (df_me.state == 'MA') | (df_me.state == 'MN') | (df_me.state == 'OH') |
               (df_me.state == 'RI')]

df_me3 = df_me2.drop_duplicates(subset='zipcode', keep='first')

df_me4 = df_me3[['zipcode','state','county','latitude','longitude','windmills_count','pop_density','med_household_income']]

df_me4 = df_me4.rename(columns = {'zipcode':'wf_zipcode'})

df_me4['zipcode_withno_wf'] = np.nan

df_me4['distance_from_wfzipcode'] = 0

df_me4.head(1)

,wf_zipcode,state,county,latitude,longitude,windmills_count,pop_density,med_household_income,zipcode_withno_wf,distance_from_wfzipcode
0,1507,MA,Worcester County,42.110889,-72.003899,1,308,93963.0,NaN,0


In [8]:
#Merge the zipcodes with windfarms and zillow dataframes on zip code, as inner join.
df_stats = pd.merge(left=df_me4, right=df_zi, on=None, left_on='wf_zipcode', right_on='ZipCode')

#Backfill years with no home value data with the closest year in the row that has data
#df_stats = df_stats.fillna(axis='columns',method='bfill')

#Only the main columns and the home value in January of years 2009 - 2018 is needed (annual change in value)
df_stats = df_stats.iloc[:, np.r_[0:13,23,35,47,59,71,83,95,107,119,131]]

#Calculate the annual change in home value as a percentage
df_stats['percent_2010_change'] = (df_stats['2010-01']/df_stats['2009-01']) - 1
df_stats['percent_2011_change'] = (df_stats['2011-01']/df_stats['2010-01']) - 1
df_stats['percent_2012_change'] = (df_stats['2012-01']/df_stats['2011-01']) - 1
df_stats['percent_2013_change'] = (df_stats['2013-01']/df_stats['2012-01']) - 1
df_stats['percent_2014_change'] = (df_stats['2014-01']/df_stats['2013-01']) - 1
df_stats['percent_2015_change'] = (df_stats['2015-01']/df_stats['2014-01']) - 1
df_stats['percent_2016_change'] = (df_stats['2016-01']/df_stats['2015-01']) - 1
df_stats['percent_2017_change'] = (df_stats['2017-01']/df_stats['2016-01']) - 1
df_stats['percent_2018_change'] = (df_stats['2018-01']/df_stats['2017-01']) - 1

df_stats.head(1)

,wf_zipcode,state,county,latitude,longitude,windmills_count,pop_density,med_household_income,zipcode_withno_wf,distance_from_wfzipcode,...,2018-01,percent_2010_change,percent_2011_change,percent_2012_change,percent_2013_change,percent_2014_change,percent_2015_change,percent_2016_change,percent_2017_change,percent_2018_change
0,1507,MA,Worcester County,42.110889,-72.003899,1,308,93963.0,NaN,0,...,276000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002946,0.013216


In [9]:
#Display a breakout by state of the number of zipcodes with windfarms
count = df_stats.state.count()
print('Total windfarms: ' + str(count))

df_states = df_stats[['state','wf_zipcode']]

state_windfarm_count = df_states.groupby(['state']).agg('count')
state_windfarm_count


Total windfarms: 46


,wf_zipcode
state,
CA,19
MA,13
MN,6
OH,5
RI,3


In [10]:
#Add a column to this dataframe that contains a value of 1 to indicate these are zipcodes with a windfarm
df_stats['Has_Windfarm'] = 1

In [11]:
#Write the zipcodes with windfarms to a csv file
#%cd C:\Users\Ken\Documents\KenP\Applications-DataScience\SpringboardCourseWork\CapstoneProject1Downloads\MachineLearningData

#out_csv = 'project1_machine_learining_zipcodes_with_windfarms.csv'

#df_stats.to_csv(out_csv)

<div class="span5 alert alert-success">
<p> Create the dataframe for zipcodes WITHOUT windfarms </p>
</div>

In [12]:
#Return at least one zipcode that is approximately 25 miles from the zipcode with a windfarm

#create empty dataframe to store zip codes with no windfarms info
df_out = df_me4.head(0)

list_zipcode_with_wf = []
list_state = []
list_county = []
list_latitude = []
list_longitude= []
list_windmills_count = []
list_pop_density = []
list_med_household_income = []
list_zipcode_withno_wf = []
list_distance_from_wfzipcode = []

for i in df_me4.index.values:
    wf_zipcode = df_me4.loc[i,'wf_zipcode']
    wf_latitude = df_me4.loc[i,'latitude']
    wf_longitude = df_me4.loc[i,'longitude']
        
    wf_zipcode_info = search.by_coordinates(wf_latitude, wf_longitude, radius=25, returns=1,sort_by='dist', ascending=False,)
        
    if len(wf_zipcode_info) != 0:
        for info in wf_zipcode_info: 
            list_zipcode_with_wf.append(info.zipcode)
            list_state.append(info.state)
            list_county.append(info.county)
            
            list_latitude.append(info.lat)
            list_longitude.append(info.lng)
            
            list_windmills_count.append(0)
            
            list_pop_density.append(info.population_density)
            list_med_household_income.append(info.median_household_income)
            
            list_zipcode_withno_wf.append(info.zipcode)
            list_distance_from_wfzipcode.append(distance(wf_latitude, wf_longitude, info.lat, info.lng))
            
    else:
        list_zipcode.append(99999)
    
df_out['wf_zipcode'] = list_zipcode_with_wf
df_out['state'] = list_state
df_out['county'] = list_county
df_out['latitude'] = list_latitude
df_out['longitude'] = list_longitude
df_out['windmills_count'] = list_windmills_count
df_out['pop_density'] = list_pop_density
df_out['med_household_income'] = list_med_household_income
df_out['zipcode_withno_wf'] = list_zipcode_withno_wf
df_out['distance_from_wfzipcode'] = list_distance_from_wfzipcode

df_out['zipcode_withno_wf'] = pd.to_numeric(df_out['zipcode_withno_wf'], errors='coerce')

df_out.head(1)

,wf_zipcode,state,county,latitude,longitude,windmills_count,pop_density,med_household_income,zipcode_withno_wf,distance_from_wfzipcode
0,01757,MA,Worcester County,42.15,-71.52,0,1886.0,66547,1757,25.006788


In [13]:
#Merge the zipcodes without windfarms and zillow dataframes on zip code, as inner join.
df_stats2 = pd.merge(left=df_out, right=df_zi, on=None, left_on='zipcode_withno_wf', right_on='ZipCode')

#Backfill years with no home value data with the closest year in the row that has data
#df_stats2 = df_stats2.fillna(axis='columns',method='bfill')

#Only the main columns and the home value in January of years 2009 - 2018 is needed (annual change in value)
df_stats2 = df_stats2.iloc[:, np.r_[0:13,23,35,47,59,71,83,95,107,119,131]]

#Calculate the annual change in home value as a percentage
df_stats2['percent_2010_change'] = (df_stats2['2010-01']/df_stats2['2009-01']) - 1
df_stats2['percent_2011_change'] = (df_stats2['2011-01']/df_stats2['2010-01']) - 1
df_stats2['percent_2012_change'] = (df_stats2['2012-01']/df_stats2['2011-01']) - 1
df_stats2['percent_2013_change'] = (df_stats2['2013-01']/df_stats2['2012-01']) - 1
df_stats2['percent_2014_change'] = (df_stats2['2014-01']/df_stats2['2013-01']) - 1
df_stats2['percent_2015_change'] = (df_stats2['2015-01']/df_stats2['2014-01']) - 1
df_stats2['percent_2016_change'] = (df_stats2['2016-01']/df_stats2['2015-01']) - 1
df_stats2['percent_2017_change'] = (df_stats2['2017-01']/df_stats2['2016-01']) - 1
df_stats2['percent_2018_change'] = (df_stats2['2018-01']/df_stats2['2017-01']) - 1

df_stats2.head(1)

,wf_zipcode,state,county,latitude,longitude,windmills_count,pop_density,med_household_income,zipcode_withno_wf,distance_from_wfzipcode,...,2018-01,percent_2010_change,percent_2011_change,percent_2012_change,percent_2013_change,percent_2014_change,percent_2015_change,percent_2016_change,percent_2017_change,percent_2018_change
0,01757,MA,Worcester County,42.15,-71.52,0,1886.0,66547,1757,25.006788,...,290300.0,NaN,NaN,NaN,NaN,0.086446,0.020857,0.02535,0.058303,0.012204


In [14]:
#Add a column to this dataframe that contains a value of 0 to indicate these are zipcodes with no windfarm
df_stats2['Has_Windfarm'] = 0

In [15]:
#Write the zipcodes with no windfarms to a csv file
#%cd C:\Users\Ken\Documents\KenP\Applications-DataScience\SpringboardCourseWork\CapstoneProject1Downloads\MachineLearningData

#out_csv = 'project1_machine_learning_zipcodes_withno_windfarms.csv'

#df_stats2.to_csv(out_csv)

<div class="span5 alert alert-success">
<p> Append the zipcodes without windfarms dataframe to the zipcodes with windfarms dataframe to create a single machine learning dataset </p>
</div>

In [16]:
#Append the zipcodes without a windfarm to the zipcodes with a windfarm to create a single dataset of zipcodes with
#and without windfarms.
df_ml = df_stats.append(df_stats2)

<div class="span5 alert alert-success">
<p> Add the feature "percent_higher_ed" to the machine learning dataset </p>
</div>

In [17]:
#Use the windfarm's zipcodes to retrieve level of education and calculate the percent of people in each zip code
#with at least a 4 year degree.  Then add this feature to the machine learning dataset.

In [18]:
#Test retrieving data using zipcode from zip code database
my_zipcode = search.by_zipcode("1757")

In [19]:
#Convert retrieved data to a dictionary
myzipdict = my_zipcode.to_dict()
#print(myzipdict)

In [20]:
#Test if education data is retrievable
'educational_attainment_for_population_25_and_over' in myzipdict

True

In [21]:
#View the education dictionary layout
my_education = myzipdict.get('educational_attainment_for_population_25_and_over')
my_education

[{'key': 'Data',
  'values': [{'x': 'Less Than High School Diploma', 'y': 1783},
   {'x': 'High School Graduate', 'y': 9043},
   {'x': "Associate's Degree", 'y': 1607},
   {'x': "Bachelor's Degree", 'y': 4443},
   {'x': "Master's Degree", 'y': 1661},
   {'x': 'Professional School Degree', 'y': 319},
   {'x': 'Doctorate Degree', 'y': 321}]}]

In [22]:
#Function to calculate the percent population of the zip code with higher education degrees
def percent_higher_ed_calc(inputlist):
    
    degree_total = 0
    edu_total = 0
    
    for k, v in enumerate(inputlist):
        list2 = v.get('values')
        
        for l,m in enumerate(list2):
            myx = m.get('x')
            myy = m.get('y')
            
            if myx != 'High School Graduate' and myx != 'Less Than High School Diploma':
                degree_total = degree_total + myy
            
            edu_total = edu_total + myy
            
        percent_higher_ed_result = degree_total/edu_total
                
        return percent_higher_ed_result

In [23]:
#Add Percent_Higher_Ed to the df_wf dataframe for zipcodes WITH A WINDFARM
df_wf_expanded = df_ml.head(46)

list1 = []
list_percent_higher_ed = []

for i in df_wf_expanded.index.values:
    wf_latitude = df_wf_expanded.loc[i,'latitude']
    wf_longitude = df_wf_expanded.loc[i,'longitude']
        
    wf_zipcode_info = search.by_coordinates(wf_latitude, wf_longitude, radius=25, returns=1)
    
    if len(wf_zipcode_info) != 0:
        for info in wf_zipcode_info:
            list1 = info.educational_attainment_for_population_25_and_over
            percent_higher_ed = percent_higher_ed_calc(list1)
            list_percent_higher_ed.append(percent_higher_ed)
    else:
        list_percent_higher_ed.append(99999)
    
df_wf_expanded['percent_higher_ed'] = list_percent_higher_ed

In [24]:
#Add Percent_Higher_Ed to the df_wf dataframe for zipcodes WITHOUT a Windfarm
df_wf_expanded2 = df_ml.tail(20)

list1 = []
list_percent_higher_ed = []

for i in df_wf_expanded2.index.values:
    wf_latitude = df_wf_expanded2.loc[i,'latitude']
    wf_longitude = df_wf_expanded2.loc[i,'longitude']
        
    wf_zipcode_info = search.by_coordinates(wf_latitude, wf_longitude, radius=25, returns=1)
    
    if len(wf_zipcode_info) != 0:
        for info in wf_zipcode_info:
            list1 = info.educational_attainment_for_population_25_and_over
            percent_higher_ed = percent_higher_ed_calc(list1)
            list_percent_higher_ed.append(percent_higher_ed)
    else:
        list_percent_higher_ed.append(99999)
    
df_wf_expanded2['percent_higher_ed'] = list_percent_higher_ed

In [25]:
#Combine into one dataframe
df_ml3 = df_wf_expanded.append(df_wf_expanded2)

#rename the housing value columns
df_ml3.rename(columns={'2009-01': 'y2009','2010-01': 'y2010', '2011-01': 'y2011','2012-01': 'y2012',
                      '2013-01': 'y2013','2014-01': 'y2014','2015-01': 'y2015','2016-01': 'y2016',
                      '2017-01': 'y2017','2018-01': 'y2018',}, inplace=True)

#delete columns not needed for machine learning
df_ml3 = df_ml3.drop(['wf_zipcode','windmills_count','zipcode_withno_wf','distance_from_wfzipcode','Unnamed: 0'], 1)

<div class="span5 alert alert-success">
<p> Write the machine learning input dataset of all years to a csv file </p>
</div>

In [26]:
%cd C:\Users\Ken\Documents\KenP\Applications-DataScience\SpringboardCourseWork\CapstoneProject1Downloads\MachineLearningData

out_csv = 'project1_machine_learning_input_all_years_dataset.csv'

df_ml3.to_csv(out_csv)

[WinError 3] The system cannot find the path specified: 'C:\\Users\\Ken\\Documents\\KenP\\Applications-DataScience\\SpringboardCourseWork\\CapstoneProject1Downloads\\MachineLearningData'
C:\Users\Ken\Documents\KenP\Applications-DataScience\SpringboardCourseWork\CapstoneProject1Repository\09 CapstoneProject1Data\MergedData


<div class="span5 alert alert-success">
<p> Create machine learning input datasets of year ranges (i.e. 2010-2017, 2011-2017, etc... to csv files </p>
</div>

In [27]:
#Some zipcodes have more years of housing data available than others.  This routine breaks the zipcodes up into groups based 
#on years of data available.

df = df_ml3.reset_index(drop=True)

#for example, df2010 contains zipcodes with annual percent change in housing value data available starting in 2010 thru 2017;
#             df2011 contains zipcodes with annual percent change in housing value data available starting in 2011 thru 2017, etc...
df2009 = df[df.y2009 > 0]

df2010 = df[df.y2010 > 0]
df2010 = df2010.drop('y2009',axis=1)
df2010 = df2010.drop('percent_2010_change',axis=1)

df2011 = df[df.y2011 > 0]
df2011 = df2011.drop(['y2009','y2010'],axis=1)
df2011 = df2011.drop(['percent_2010_change','percent_2011_change'],axis=1)

df2012 = df[df.y2012 > 0]
df2012 = df2012.drop(['y2009','y2010','y2011'],axis=1)
df2012 = df2012.drop(['percent_2010_change','percent_2011_change','percent_2012_change'],axis=1)

df2013 = df[df.y2013 > 0]
df2013 = df2013.drop(['y2009','y2010','y2011','y2012'],axis=1)
df2013 = df2013.drop(['percent_2010_change','percent_2011_change','percent_2012_change','percent_2013_change'],axis=1)

df2014 = df[df.y2014 > 0]
df2014 = df2014.drop(['y2009','y2010','y2011','y2012','y2013'],axis=1)
df2014 = df2014.drop(['percent_2010_change','percent_2011_change','percent_2012_change','percent_2013_change',
                      'percent_2014_change'],axis=1)

df2015 = df[df.y2015 > 0]
df2015 = df2015.drop(['y2009','y2010','y2011','y2012','y2013','y2014'],axis=1)
df2015 = df2015.drop(['percent_2010_change','percent_2011_change','percent_2012_change','percent_2013_change',
                      'percent_2014_change','percent_2015_change'],axis=1)

In [28]:
#Write dataframes out to csv files
out_csv = 'project1_machine_learning_input_starting_2009.csv'
df2009.to_csv(out_csv)

out_csv = 'project1_machine_learning_input_starting_2010.csv'
df2010.to_csv(out_csv)

out_csv = 'project1_machine_learning_input_starting_2011.csv'
df2011.to_csv(out_csv)

out_csv = 'project1_machine_learning_input_starting_2012.csv'
df2012.to_csv(out_csv)

out_csv = 'project1_machine_learning_input_starting_2013.csv'
df2013.to_csv(out_csv)

out_csv = 'project1_machine_learning_input_starting_2014.csv'
df2014.to_csv(out_csv)

out_csv = 'project1_machine_learning_input_starting_2015.csv'
df2015.to_csv(out_csv)